In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import gc #garbage collection
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBRFClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, roc_auc_score
import datatable as dt

In [ ]:
training = True
param_tuning = False

In [ ]:
%%time
# Load data
train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
train_data = train_data_datatable.to_pandas()

In [ ]:
del train_data_datatable
gc.collect()

In [ ]:
label = 'action'
features = [c for c in train_data.columns if 'feature' in c]

train_data['action'] = ((train_data['resp'].values) > 0).astype('int') 
# inlude weight train['weight'].values? non-zero threshold for resp?

# # Use trade id as index
# train_data.set_index(['ts_id'],drop=True)
train_data.shape

In [ ]:
%%time
# fill nan with feature mean
feature_means = train_data[features].mean()
train_data.fillna(feature_means,inplace=True)

In [ ]:
# first 85 days excluded
train_data = train_data[train_data['date']>85]
train_data.reset_index(drop=True,inplace=True)

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]
            
            


In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
from matplotlib.colors import ListedColormap

def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    
    cmap_cv = plt.cm.coolwarm

    jet = plt.cm.get_cmap('jet', 256)
    seq = np.linspace(0, 1, 256)
    _ = np.random.shuffle(seq)   # inplace
    cmap_data = ListedColormap(jet(seq))

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['target', 'day']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

In [ ]:
cv = GroupTimeSeriesSplit()

In [ ]:
# plot GroupTimeSeriesSplit

# fig, ax = plt.subplots()

# plot_cv_indices(
#     cv,
#     X=train_data.loc[train_data['date'] <100,features].values,
#     y=train_data.loc[train_data['date'] <100,label].values,
#     group =train_data.loc[train_data['date']<100,'date'].values,
#     ax=ax,
#     n_splits = 5,
#     lw=20
# )

In [ ]:
# Tuning with hyperopt

# Cross Val score as function to be optimised
def score(params):
    
    ps = {'learning_rate': params['learning_rate'],
         'max_depth': params['max_depth'], 
         'gamma': params['gamma'], 
         'min_child_weight': params['min_child_weight'], 
         'subsample': params['subsample'], 
         'colsample_bytree': params['colsample_bytree'],
          'random_state': 0,
#          'verbosity': 1, 
         'objective': 'binary:logistic',
         'eval_metric': 'auc', 
         'tree_method': 'gpu_hist' 
        }
    
    X = train_data[features]
    y = train_data[label]
    groups = train_data['date'].values
    
    scores = []
    for i, (train_idx, valid_idx) in enumerate(cv.split(
        X = X.values,
        y = y.values,
        groups=groups)):
        
        d_tr = xgb.DMatrix(X.iloc[train_idx], y.iloc[train_idx])
        d_val = xgb.DMatrix(X.iloc[valid_idx,:], y.iloc[valid_idx])
        
        model = xgb.train(ps,d_tr, 
                          1000,#params['n_round'],
                          [(d_val, 'eval')],
                          early_stopping_rounds=10,
                          verbose_eval = False)
        y_pred = model.predict(d_val)
        score = roc_auc_score(y.iloc[valid_idx], y_pred)
        scores.append(score)
   
    final_score = np.average(scores) * -1.0

    return final_score

In [ ]:
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope

param_space = {'learning_rate': hp.uniform('learning_rate', 0.01, 0.3), 
               'max_depth': scope.int(hp.quniform('max_depth', 5, 13, 1)), 
               'gamma': hp.uniform('gamma', 0, 10), 
               'min_child_weight': hp.uniform('min_child_weight', 0, 5),
               'subsample': hp.uniform('subsample', 0.3, 1), 
               'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1), 
#                'n_round': scope.int(hp.quniform('n_round', 25, 1000, 25))
               #scope.int(hp.quniform('n_round', 3, 8, 1))  
              }

In [ ]:
%%time
# Run optimiser
if(param_tuning):
    max_evals = 50

    trials = Trials()

    hopt = fmin(fn = score,
                space = param_space, 
                algo = tpe.suggest, 
                max_evals = max_evals, ## 50
                trials = trials, 
               )

In [ ]:
# plot parameter choice in trials for selected parameters
if(param_tuning):
    params_to_plot = ['learning_rate', 'max_depth', 'min_child_weight','gamma']
    fig, ax = plt.subplots(len(params_to_plot),1, figsize= (25,20))
    fig.subplots_adjust(hspace=0.5)
    for i,param in enumerate(params_to_plot):
        xs = [t['tid'] for t in trials.trials]
        ys = [t['misc']['vals'][param] for t in trials.trials]
        ax[i].scatter(xs, ys, s=20, linewidth=0.01, alpha=0.75)
        ax[i].set_title(str(param) + ' vs t ', fontsize=18)
        ax[i].set_xlabel('id', fontsize=16)
        ax[i].set_ylabel(str(param), fontsize=16)

In [ ]:
if(param_tuning):
    f, ax = plt.subplots(1)
    xs = [t['tid'] for t in trials.trials]
    ys = [t['result']['loss'] for t in trials.trials]
    ax.scatter(xs, ys, s=20, linewidth=0.01, alpha=0.75)
    ax.set_title('loss over time', fontsize=18)
    ax.set_xlabel('trials', fontsize=16)
    ax.set_ylabel('loss', fontsize=16)

In [ ]:
# hopt

In [ ]:
# d_tr = xgb.DMatrix(train_data[features], train_data[label])

In [ ]:
if(training):
    X_train, X_val, y_train, y_val = train_test_split(
        train_data[features], train_data[label],test_size = 0.2,shuffle=False)
    d_tr = xgb.DMatrix(X_train,y_train)
    d_val = xgb.DMatrix(X_val,y_val)

In [ ]:
del train_data
gc.collect()

In [ ]:
%%time
# Tuned XGBoost


p_tuned = {'learning_rate': 0.015,
          'max_depth': 8, 
          'gamma': 10, 
          'min_child_weight': 0.3, 
          'subsample': 0.5, 
          'colsample_bytree': 1, 
          'random_state' : 1987,
          'objective': 'binary:logistic',
          'eval_metric': 'auc', 
          'tree_method': 'gpu_hist',
         }

# p_tuned = {'learning_rate': 0.019873752423708882,
#           'max_depth': 11, 
#           'gamma': 8.840988783928074, 
#           'min_child_weight': 1.5115254729441987, 
#           'subsample': 0.4743001776441187, 
#           'colsample_bytree': 0.45881171645881513, 
#           'random_state' : 1987,
#           'objective': 'binary:logistic',
#           'eval_metric': 'auc', 
#           'tree_method': 'gpu_hist',
#          }

if training:
    progress = dict()
#     watchlist  = [(train,'train-auc'), (eval, 'eval-auc')]
    clf = xgb.train(p_tuned,
                    d_tr,
                    num_boost_round = 2000,
                    evals = [(d_tr,'train'),(d_val,'eval')],
                    early_stopping_rounds= 25,
#                     watchlist, 
                    evals_result=progress,
                    verbose_eval = 100)

In [ ]:
x = np.arange(0,len(progress['train']['auc']))
plt.plot(x,progress['train']['auc'], label ='train-auc')
plt.plot(x,progress['eval']['auc'], label = 'eval-auc')
plt.legend()

In [ ]:
if training:
    clf.save_model(f'es_xgb_model.model')

In [ ]:
if training:
    model = clf
else:
    model = xgb.Booster()  # init model
    model.load_model('/kaggle/input/xgbjanestreet/es2_xgb_model.model')  # load data

In [ ]:
%%time
clf_preds = model.predict(d_tr)

In [ ]:
# %%time
# # Load data again for eval function
# train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
# train_data = train_data_datatable.to_pandas()
# del train_data_datatable
# gc.collect()

In [ ]:
# label = 'action'
# features = [c for c in train_data.columns if 'feature' in c]

# train_data['action'] = ((train_data['resp'].values) > 0).astype('int') 
# train_data.shape

In [ ]:
# # code up the evaluation function
# def evaluate_util(X):
#     grouped = X.groupby('date')
#     ## depends on definition of action ##
#     pi = grouped.apply(lambda x : np.sum(x['weight'] * x['resp'] * x['action_pred']) )
#     t = np.sum(pi)/np.sqrt(np.sum(np.power(pi,2))) * np.sqrt(250)/len(pi)
#     u = np.minimum(np.maximum(t,0),6) * np.sum(pi)
#     return u
# # evaluate_util(train)

In [ ]:
# train_data['action_pred'] = train_data['action'] 
# evaluate_util(train_data)

In [ ]:
# d_tr = xgb.DMatrix(train_data[features], train_data[label])
# clf_preds = model.predict(d_tr)

In [ ]:
# _, X_val, _, y_val = train_test_split(train_data[features],train_data[label],test_size = 0.2,shuffle=False)

In [ ]:
# d_val = xgb.DMatrix(X_val, y_val)
# clf_val_preds = model.predict(d_val)

In [ ]:
# val_preds = np.where(clf_val_preds >= 0.5,1,0)

In [ ]:
# roc_auc_score(y_val,val_preds)

In [ ]:
# del d_tr
# gc.collect()

In [ ]:
# %%time
# # check optimal threshold
# thresholds = [0.475 + i/1000 for i in range(0,50)]
# eval_scores = {}
# for th in thresholds:
#     train_data['action_pred'] = np.where(clf_preds >= th, 1, 0).astype(int)
#     eval_scores[th] = evaluate_util(train_data)
# th_opt = max(eval_scores, key=eval_scores.get)
# print(th_opt)
# print(max(eval_scores.values()))
# scores = [x for x in eval_scores.values()]
# plt.plot(thresholds,scores)
# plt.show()

In [ ]:
# del X_train, X_val, y_train, y_val
# gc.collect()

In [ ]:
# prepare fill function
from numba import njit
@njit
def fast_fillna(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

In [ ]:
# check if it's working
example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
example_test[features] = example_test[features].fillna(method = 'ffill').fillna(0)
d_tt = xgb.DMatrix(example_test[features].values)
model.predict(d_tt,validate_features=False )

In [ ]:
# Jane Street submission environment
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
# sample_prediction_df.action = 0
# env.predict(sample_prediction_df)

In [ ]:
%%time
th_opt = 0.501

for (test_df, sample_prediction_df) in iter_test:   
    if test_df['weight'].item() > 0:
#         x_tst = test_df.loc[:, test_df.columns.str.contains('feature')]
#         x_tst.iloc[0,:] = fast_fillna(x_tst.iloc[0,:].values, feature_means.values)
        x_tst = test_df.loc[:, features].values
        x_tst[0,:] = fast_fillna(x_tst[0,:], feature_means.values)
        d_tst = xgb.DMatrix(x_tst)
        pred = clf.predict(d_tst,validate_features=False)
        sample_prediction_df.action = np.where(pred >= th_opt, 1, 0).astype(int)
    else:
        sample_prediction_df.action = 0
    env.predict(sample_prediction_df)
        
#     X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
#     for i in X_test.index:   
#         X_test.loc[i,features] = fast_fillna(X_test.loc[i,features],feature_means)
#     d_tst = xgb.DMatrix(X_test)
#     y_preds = clf.predict(d_tst)
#     sample_prediction_df.action = y_preds
#     env.predict(sample_prediction_df)

In [ ]:
# # random submission for testing
# # import random
# # import janestreet
# # env = janestreet.make_env() # initialize the environment
# # iter_test = env.iter_test() # an iterator which loops over the test set

# for (test_df, sample_prediction_df) in iter_test:
#     sample_prediction_df.action = random.randint(0, 1) #make your 0/1 prediction here
#     env.predict(sample_prediction_df)